## 1. Importing libraries

In [1]:
#importing required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


## 2. Importing datasets

In [2]:
#Reading in data 
## Testing data

#Confirmed provincial cummulative  tests         
df5 = pd.read_csv('./../data/raw/owid-covid-data.csv', sep= ',')
sa_cumm=df5[df5['location'] == 'South Africa']

#Confirmed national cummulative confirmed positive tests         
df6 = pd.read_csv('./../data/raw/time_series_covid19_confirmed_global.csv', sep= ',')
sa_con=df6[df6['Country/Region'] == 'South Africa']

#Confirmed national cummulative confirmed deaths         
df7 = pd.read_csv('./../data/raw/time_series_covid19_deaths_global.csv', sep= ',')
sa_death=df7[df7['Country/Region'] == 'South Africa']

#Confirmed national cummulative confirmed revovered tests         
df8 = pd.read_csv('./../data/raw/time_series_covid19_recovered_global.csv', sep= ',')
sa_recovd=df8[df8['Country/Region'] == 'South Africa']


## Hospital data

#Number of hospital beds per province
hosp_beds = pd.read_csv('./../data/raw/hospital_beds.csv')

#List of hospitals in each povince
url="https://raw.githubusercontent.com/dsfsi/covid19za/master/data/health_system_za_public_hospitals.csv"
hosp_list=pd.read_csv(url)

In [55]:
#renaming columns

sa_con.rename(columns={'cases':'confirmed_cases'}, inplace=True)
sa_death.rename(columns={'cases':'death_cases'}, inplace=True)
sa_recovd.rename(columns={'cases':'recovered_cases'}, inplace=True)

In [56]:
sa_death.head(1)

,country,date,death_cases
0,South Africa,2020-01-22,0


In [50]:
df_merged=pd.merge(sa_con, sa_death, how='left')
df_merged

,country,date,cases
0,South Africa,2020-01-22,0
1,South Africa,2020-01-23,0
2,South Africa,2020-01-24,0
3,South Africa,2020-01-25,0
4,South Africa,2020-01-26,0
5,South Africa,2020-01-27,0
6,South Africa,2020-01-28,0
7,South Africa,2020-01-29,0
8,South Africa,2020-01-30,0
9,South Africa,2020-01-31,0


In [ ]:
# Merging testing files
sa_cumm
sa_con
sa_death
sa_recovd

#Merging hospital data
hosp_beds
hosp_list

## 3.EDA

In [4]:
#Dropping unwanted columns

#List of hospitals
hosp_list = hosp_list.drop(['ID','Lat', 'Long','district','district_estimated_population','subdistrict', 'service_offered_by_hospital', 'size_hospital','number_of_beds','number_of_practitioners','webpage','geo_subdivision'], axis=1)

#Confirmed national cummulative tests conducted
sa_cumm = sa_cumm.drop(['iso_code'], axis=1)

#Confirmed national cummulative confirmed positive tests 
sa_con = sa_con.drop(['Lat', 'Long','Province/State'], axis=1)

#Confirmed national cummulative confirmed deaths 
sa_death = sa_death.drop(['Lat', 'Long','Province/State'], axis=1)

#Confirmed national cummulative confirmed recovered
sa_recovd = sa_recovd.drop(['Lat', 'Long','Province/State'], axis=1)

In [6]:
#Cleaning up columns

#Creating date column for sa_con dataframe
dfc1 = sa_con.melt(id_vars=["Country/Region"], 
                    var_name="Date",
                    value_name="cases")
dfc1 = dfc1.rename({'Country/Region': 'country', 'Date': 'date'}, axis=1)
sa_con = dfc1
sa_con['date'] = pd.to_datetime(sa_con['date'])

#Creating date column for sa_death dataframe
dfc2 = sa_death.melt(id_vars=["Country/Region"], 
                    var_name="Date",
                    value_name="cases")
dfc2 = dfc2.rename({'Country/Region': 'country', 'Date': 'date'}, axis=1)
sa_death = dfc2
sa_death['date'] = pd.to_datetime(sa_death['date'])

#Creating date column for sa_recovd dataframe
dfc3 = sa_recovd.melt(id_vars=["Country/Region"], 
                    var_name="Date",
                    value_name="cases")
dfc3 = dfc3.rename({'Country/Region': 'country', 'Date': 'date'}, axis=1)
sa_recovd = dfc3
sa_recovd['date'] = pd.to_datetime(sa_recovd['date'])


##  4.  Statistical analysis

### 4.1 Testing dataset

In [44]:
#Looking closely into the Confirmed provincial cummulative dataset

# display of latest figures
totl_cases = sa_cumm['total_cases'].iloc[-1]
tot_deaths = sa_cumm['new_cases'].iloc[-1]
date = sa_cumm['date'].iloc[-1]

print('Date:{} Total Cases:{} Fatalities:{}' .format(date,totl_cases,tot_deaths), sep='nl')


#Display of the cummulative results


Date:2020-05-11 Total Cases:10015 Fatalities:595
